In [18]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset

In [ ]:
#TODO: This is not using all the features in the dataset
def create_model(df):
    dataset = Dataset()
    dataset.fit(df['user_id'].unique(), df['song_id'].unique())

    # Play count is string in the dataset for some reason
    df['play_count'] = pd.to_numeric(df['play_count'])
    
    interactions = dataset.build_interactions(
        ((row['user_id'], row['song_id'], row['play_count']) 
         for idx, row in df.iterrows())
    )[0]
    
    model = LightFM(loss='warp')
    model.fit(interactions, epochs=30, num_threads=2)
    
    return model, dataset

def get_recommendations(model, dataset, df, user_index, num_recommendations=5):
    target_user_id = df['user_id'].unique()[user_index]
    n_items = len(dataset.mapping()[2])
    user_id_map = dataset.mapping()[0]
    user_internal_id = user_id_map[target_user_id]
    
    scores = model.predict(user_internal_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:num_recommendations]
    
    item_map = {v: k for k, v in dataset.mapping()[2].items()}
    return [item_map[i] for i in top_items], target_user_id

In [20]:
df = pd.read_csv('song_dataset.csv', header=None, 
                 names=['user_id', 'song_id', 'play_count', 'title', 'album', 'artist', 'year'])

df = df[1:]


In [21]:
# Building the interactions matrix https://making.lyst.com/lightfm/docs/lightfm.data.html
# Some play_count values are missing, so we'll fill them with 0
# and convert the column to numeric because it's currently a string
'''TODO: Decide what we do with string values'''
create_model(df)
model, dataset = create_model(df)


In [24]:
USER_INDEX = 1
recommendations, target_user_id = get_recommendations(model, dataset, df, USER_INDEX)
print(f"Recommendations for user {target_user_id}:")
for song_id in recommendations:
    print(df[df['song_id'] == song_id]['title'].values[0] + ' by ' + df[df['song_id'] == song_id]['artist'].values[0])

Recommendations for user 85c1f87fea955d09b4bec2e36aee110927aedf9a:
We're Not Gonna Take It by Less Than Jake
Disco Infiltrator (FK's Infiltrated Vocal) by LCD Soundsystem
Signos by Soda Stereo
Main Title by Alan Silvestri
Super Freak by Rick James
